In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl 
import os
import glob
import datetime
import openpyxl
from openpyxl.styles import colors, Font, Color, Border, Side, Alignment, PatternFill
%matplotlib inline

In [2]:
INX_NAME_WORK_LOAD = 4
INX_NAME_DETAIL_COST = 1
SH_COMPANY = ['上海']
GD_COMPANY = ['广上', '广州', '广深']
NJ_COMPANY = ['南京']

In [3]:
work_load_file = r'.\工时分摊表202101_1.xlsx'
work_load_sheet = '202101调整'
detail_cost_file = r'.\部门费用.xlsx'
detail_cost_sheet = '日常费用'

# work_load_file = r'.\2020人员工时分摊_12月.xlsx'
# work_load_sheet = '2020预算'
# detail_cost_file = r'.\工时分摊表资料模板.xlsx'
# detail_cost_sheet = '2020年终预算估'

xls_file_name = '%s汇总_%s.xlsx' % (detail_cost_sheet, datetime.datetime.now().date().strftime('%y%m%d'))
xlsx=pd.ExcelWriter(xls_file_name)

In [4]:
def clean_work_load(df_work_load):
    valid_columns = df_work_load.columns.values[INX_NAME_WORK_LOAD-1:-2]
    df = df_work_load[valid_columns]
    df = df.set_index(valid_columns[2])
    df = df.fillna(0)
    index_name=df.index.name[0]
    df.index.name=index_name
    return df


def clean_detail_cost(df_detail_cost):
    # valid_columns = df_detail_cost.columns.values[INX_NAME_DETAIL_COST:-1]
    valid_columns = df_detail_cost.columns.values[INX_NAME_DETAIL_COST:]
    df = df_detail_cost[valid_columns]
    df = df.set_index(valid_columns[0])
    df = df.fillna(0)
    return df

def check_data(df_work_load, df_detail_cost):
    df = df_work_load.copy()
    col_attr=df.columns[0]
    s=(df[col_attr]=='研发') | (df[col_attr]=='非研发')
    if s.all()==False:
        txt = '<工时比例>：属性只能为“研发” “非研发”'
        raise Exception(txt)

    col_attr = df.columns[1]
    valid_val=set(SH_COMPANY+GD_COMPANY+NJ_COMPANY)
    all_val = set(df[col_attr].tolist())
    if (valid_val | all_val) != valid_val:
        txt = '<工时比例>：支付归口不为%s' % valid_val
        raise Exception(txt)

    df1 = df.iloc[:, 2:]
    df1 = df1.select_dtypes(exclude=['float64', 'int64'])
    if not df1.empty:
        txt = df1.columns.values
        raise Exception('<工时比例>%s: 有非法值(not float64/int64)' % txt)
        # return '<工时比例>%s: 有非法值(not float64)' % txt

    df1 = df_work_load.sum(axis=1)
    criteria = (df1 < 0.9999) | (df1 > 1.00001)
    if criteria.any():
        txt = df1[criteria].index.values
        raise Exception('<工时比例>%s: 合计不是100%%' % txt)

    col_attr = df.columns[0]
    df1 = df[df[col_attr] != '研发'].iloc[:, -1:]
    criteria = (df1 < 0.9999) | (df1 > 1.00001)
    s_criteria = criteria.T.iloc[0]
    if s_criteria.any():
        txt = df1[s_criteria].index.values
        raise Exception('<工时比例>%s: 非研发类“其他”不为100%%' % txt)

    col_attr = df.columns[0]
    df1 = df[df[col_attr] == '研发'].iloc[:, -1:]
    criteria = (df1 < -0.00001) | (df1 > 0.00001)
    s_criteria = criteria.T.iloc[0]
    if s_criteria.any():
        txt = df1[s_criteria].index.values
        raise Exception('<工时比例>%s: 研发类“其他”不为0%%' % txt)

    df = df_work_load.copy()
    col_attr = df.columns[1]
    loc_txt='上海'
    for loc in SH_COMPANY:
        df1 = df[df[col_attr] == loc]
        df2=df1.iloc[:,:-1]
        sr=df2.stack([0, 1, 2])
        sr1=sr.loc[(slice(None),slice(None),slice(None),['广东', '南京'])]
        criteria = (sr1 < -0.00001) | (sr1 > 0.00001)
        if criteria.any():
            txt = sr1[criteria].index.values
            raise Exception('<工时比例>%s: 非"%s"项目不为0' % (txt, loc_txt))
    loc_txt='广东'
    for loc in GD_COMPANY:
        df1 = df[df[col_attr] == loc]
        df2=df1.iloc[:,:-1]
        sr=df2.stack([0, 1, 2])
        sr1=sr.loc[(slice(None),slice(None),slice(None),['上海', '南京'])]
        criteria = (sr1 < -0.00001) | (sr1 > 0.00001)
        if criteria.any():
            txt = sr1[criteria].index.values
            raise Exception('<工时比例>%s: 非"%s"项目不为0' % (txt, loc_txt))
    loc_txt='南京'
    for loc in NJ_COMPANY:
        df1 = df[df[col_attr] == loc]
        df2=df1.iloc[:,:-1]
        sr=df2.stack([0, 1, 2])
        sr1=sr.loc[(slice(None),slice(None),slice(None),['上海', '广东'])]
        criteria = (sr1 < -0.00001) | (sr1 > 0.00001)
        if criteria.any():
            txt = sr1[criteria].index.values
            raise Exception('<工时比例>%s: 非"%s"项目不为0' % (txt, loc_txt))

    df = df_detail_cost.copy()
    s1 = df.index.names != ['部门']
    s2 = df.columns.to_list()[0:2] != ['属性', '支付归口']
    if s1 or s2:
        txt = '<部门费用>：列标题应该为“部门” “属性” “支付归口”'
        raise Exception(txt)

    df1 = df.iloc[:, 2:]
    df1 = df1.select_dtypes(exclude=['float64', 'int64'])
    if not df1.empty:
        txt = df1.columns.values
        raise Exception('<部门费用>%s: 有非法值(not float64/int64)' % txt)

    col_attr = df.columns[0]
    s = (df[col_attr] == '研发') | (df[col_attr] == '非研发')
    if s.all() == False:
        txt = '<部门费用>：属性只能为“研发” “非研发”'
        raise Exception(txt)

    col_attr = df.columns[1]
    valid_val=set(SH_COMPANY+GD_COMPANY+NJ_COMPANY)
    all_val = set(df[col_attr].tolist())
    if (valid_val | all_val) != valid_val:
        txt = '<部门费用>：支付归口不为%s' % valid_val
        raise Exception(txt)

def adj_ratio_by_proj_loc(df_work_load, loc_txt):
    df = df_work_load.iloc[:, 2:-1]
    sr = df.stack([0, 1, 2])
    sr.index.names=['人员','大类', '项目', '项目归属']
    sr=sr.groupby(['大类', '项目', '项目归属']).sum()
    sum_proj_loc=sr.groupby(['项目归属']).sum().sum()
    ratio_by_proj_loc=sr.groupby(['项目归属']).sum()[loc_txt]/sum_proj_loc
    ratio_by_proj_loc=sr.groupby(['项目归属']).sum()[loc_txt]
    return sr.loc[(slice(None), slice(None), [loc_txt])].mul(1/ratio_by_proj_loc)

def entire_cost(df_work_load, df_detail_cost):
    df = df_detail_cost.copy()
    df = df.reset_index()
    df = df.set_index(['部门', '属性', '支付归口'])

    index_v = df.index.values

    new_index = []
    new_val=[]
    for idx in index_v:
        dept=idx[0]
        loc = idx[2]
        if idx[1]=='非研发':
            val=df.loc[idx].values
            # dept=idx[0]
            # loc = idx[2]
            family='其他'
            project='0000'
            proj_loc = ''
            new_idx=(dept,loc,family,project, proj_loc)
            new_index.append(new_idx)
            new_val.append(val)
            
            # print(val)
        else:
            if loc in SH_COMPANY:
                loc_txt='上海'
            elif loc in GD_COMPANY:
                loc_txt='广东'
            else:
                loc_txt='南京'

            sr = adj_ratio_by_proj_loc(df_work_load, loc_txt)
            for idx_sr in sr.index.values:
                # print(idx_sr)
                ratio=sr[idx_sr]
                # print(df.loc[idx]*ratio)
                family=idx_sr[0]
                project=idx_sr[1]
                proj_loc = idx_sr[2]
                val=df.loc[idx].values*ratio
                # print(val)
                new_idx=(dept,loc,family,project, proj_loc)
                new_index.append(new_idx)
                new_val.append(val)
                
    index = pd.MultiIndex.from_tuples(new_index)
    df_new = pd.DataFrame(new_val, index=index, columns=df.columns)
    df_new.index.names=['部门', '属地', '大类', '项目', '项目归属']
    return df_new

def format_xlsx(workbook, sheet_name):
    ws = workbook[sheet_name]

    font = Font(name='Arial', size=10, color=colors.BLACK)
    hd_font = Font(name='Arial', size=10, color=colors.BLACK, bold=True)
    thin = Side(border_style="thin")
    border = Border(top=thin, left=thin, right=thin, bottom=thin)
    alignment = Alignment(horizontal="center", vertical="center")
    fill = PatternFill(fill_type="solid", fgColor="C1CDC1")

    nrows = ws.max_row
    ncols = ws.max_column

    for i in range(nrows):
        for j in range(ncols):
            cell = ws.cell(i+1, j+1)
            cell.number_format = '0.00'
            cell.border = border
            cell.alignment = alignment
            if(cell.font.b):
                cell.font = hd_font
                cell.fill = fill
            else:
                cell.font = font


def write_xlsx(df, xlsx, title, month):
    sheet_name = '%s-%s' % (month, title)
    df.to_excel(xlsx, sheet_name=sheet_name, merge_cells=False)

In [5]:
df_work_load = pd.read_excel(work_load_file, sheet_name=work_load_sheet,  header=[0, 1, 2])
df_detail_cost = pd.read_excel(detail_cost_file, sheet_name=detail_cost_sheet)
df_work_load = clean_work_load(df_work_load)
df_detail_cost = clean_detail_cost(df_detail_cost)

In [6]:
check_data(df_work_load, df_detail_cost)

In [7]:
title = '合并'
df = entire_cost(df_work_load, df_detail_cost)
write_xlsx(df, xlsx, title, month=work_load_sheet)

title = '上海'
df1=df.loc[(slice(None), SH_COMPANY), :]
write_xlsx(df1, xlsx, title, month=work_load_sheet)

title = '广东'
df2=df.loc[(slice(None), GD_COMPANY), :]
write_xlsx(df2, xlsx, title, month=work_load_sheet)

title = '南京'
df3=df.loc[(slice(None), NJ_COMPANY), :]
write_xlsx(df3, xlsx, title, month=work_load_sheet)

xlsx.close()

workbook = openpyxl.load_workbook(xls_file_name)
for sht in workbook.sheetnames:
    format_xlsx(workbook, sheet_name=sht)
workbook.save(filename=xls_file_name)

In [8]:
df.loc[(slice(None), GD_COMPANY), :]

生日福利  办公硬件        商业保险   12月份顺丰快递   12月份中通快递  \
部门    属地 大类     项目   项目归属                                                   
总经办   广州 其他     0000       100.00   0.0    0.000000   0.000000    0.00000   
技术中心  广州 家用电器   A780 广东      0.00   0.0  122.285250  32.775000  100.05000   
         通用和遥控器 A880 广东      0.00   0.0   62.914875  16.862500   51.47500   
                B010 广东      0.00   0.0   19.494750   5.225000   15.95000   
         锂电数码   A860 广东      0.00   0.0   78.865125  21.137500   64.52500   
技术服务部 广州 家用电器   A780 广东    172.50   0.0  179.339625   9.918750   19.83750   
         通用和遥控器 A880 广东     88.75   0.0   92.268938   5.103125   10.20625   
                B010 广东     27.50   0.0   28.590375   1.581250    3.16250   
         锂电数码   A860 广东    111.25   0.0  115.661063   6.396875   12.79375   
测试部   广州 家用电器   A780 广东      0.00   0.0    0.000000  25.012500    0.00000   
         通用和遥控器 A880 广东      0.00   0.0    0.000000  12.868750    0.00000   
                B010 广东      0.00   0.0    0.000000   3.987500    0.00000   
         锂电数码   A860 广东      0.00   0.0    0.000000  16.131250    0.00000   
人事行政部 广州 其他     0000         0.00   0.0  253.140000   0.000000    0.00000   
质量部   广州 家用电器   A780 广东      0.00   0.0    0.000000  12.075000    2.58750   
         通用和遥控器 A880 广东      0.00   0.0    0.000000   6.212500    1.33125   
                B010 广东      0.00   0.0    0.000000   1.925000    0.41250   
         锂电数码   A860 广东      0.00   0.0    0.000000   7.787500    1.66875   

                           11月份中通快递  
部门    属地 大类     项目   项目归属            
总经办   广州 其他     0000        0.00000  
技术中心  广州 家用电器   A780 广东    95.73750  
         通用和遥控器 A880 广东    49.25625  
                B010 广东    15.26250  
         锂电数码   A860 广东    61.74375  
技术服务部 广州 家用电器   A780 广东    11.21250  
         通用和遥控器 A880 广东     5.76875  
                B010 广东     1.78750  
         锂电数码   A860 广东     7.23125  
测试部   广州 家用电器   A780 广东     0.00000  
         通用和遥控器 A880 广东     0.00000  
                B010 广东     0.00000  
         锂电数码   A860 广东     0.00000  
人事行政部 广州 其他     0000        0.00000  
质量部   广州 家用电器   A780 广东     2.58750  
         通用和遥控器 A880 广东     1.33125  
                B010 广东     0.41250  
         锂电数码   A860 广东     1.66875